## To Create the First Dataframe

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

To download data and read the table

In [3]:
df = pd.read_html(url)[0]

Ignore cells with a borough that is Not assigned.

In [4]:
df=df[~df['Borough'].isin(['Not assigned'])]

More than one neighborhood can exist in one postal code area.   
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [5]:
def jjv(df):
    return','.join(df.values)
df2 = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(jjv).reset_index()
df2.Neighbourhood.replace('Not assigned',df2.Borough, inplace=True)

To check the dataframe

In [6]:
df2.rename(columns={'Postcode':'PostalCode'}, inplace = True)
df2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
df2.shape

(103, 3)

## To Create the Second Dataframe(geographical coordinates)

In [8]:
df_g=pd.read_csv('http://cocl.us/Geospatial_data')

In [9]:
df_g.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
df_g.rename(columns={'Postal Code':'PostalCode'}, inplace = True)

In [11]:
df_all = pd.merge(df2, df_g)

In [12]:
df_all.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
df_all.shape

(103, 5)

## Explore and cluster the neighborhoods in Toronto.

#### Create a map with neighborhoods superimposed on top.

In [15]:
#!conda install -c conda-forge geopy --yes

Solving environment: done

## Package Plan ##

  environment location: /root/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py37_0         147 KB  conda-forge
    conda-4.7.12               |           py37_0         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.20.0-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi                                

In [17]:
#!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /root/anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py37_0         773 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         871 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/linux-64::altair-3.2.0-py37_0
  branca             conda-forge/noarch::branca-0.3.1-py_0
  folium             cond

In [18]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [27]:
latitude = 43.806686
longitude = -79.194353

In [29]:
map_toro = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_all['Latitude'], df_all['Longitude'], df_all['Borough'], df_all['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toro)  
    
map_toro

#### Run *k*-means to cluster the neighborhood into 5 clusters.

In [32]:
df_c = df_all[['Latitude','Longitude']]

In [34]:
from sklearn.cluster import KMeans
kclusters = 5
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_c)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 2, 0, 2], dtype=int32)